# Referências da Documentação AWS:

- https://docs.aws.amazon.com/bedrock/latest/userguide/inference-invoke.html

- https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html

- hytps://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/invoke_model.html

# Dependências

In [ ]:
import json
import os
from datetime import datetime, timezone

import boto3

# Constantes

In [ ]:
# Pasta local para salvar respostas das chamadas AWS
OUTPUT_FOLDER = "respostas"

# ID do modelo Amazon Nova Lite (modelo de linguagem da AWS)
MODEL_ID = 'amazon.nova-lite-v1:0'

# Criar pasta local para respostas

In [ ]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Configurações do AWS BEDROCK

In [ ]:
# Inicializa o cliente do Bedrock Runtime para fazer chamadas à API
bedrock_runtime_client = boto3.client('bedrock-runtime')

# Configuração da requisição

In [ ]:
# Define a pergunta que será enviada ao modelo
prompt = "Resuma a história do Brasil em 4 parágrafos."

# Estrutura da requisição seguindo o formato messages-v1 do Bedrock
body = {
    "messages": [
        {
            "role": "user",           # Papel do remetente da mensagem
            "content": [{"text": prompt}]  # Conteúdo da mensagem
        }
    ],
    "inferenceConfig": {
        "maxTokens": 400,    # Máximo de tokens na resposta (controla o tamanho)
        "temperature": 0.7   # Criatividade da resposta (0.0 = conservador, 1.0 = criativo)
    }
}

# Teste 1: invoke_model 

In [ ]:
# Este método retorna a resposta completa de uma só vez (não streaming)
response = bedrock_runtime_client.invoke_model(
    body=json.dumps(body),              # Converte o corpo para JSON
    contentType='application/json',      # Tipo de conteúdo da requisição
    accept='application/json',           # Tipo de resposta aceito
    modelId=MODEL_ID                    # ID do modelo a ser usado
)

In [ ]:
# Salva a resposta completa (metadados + conteúdo) em arquivo JSON
with open(f'{OUTPUT_FOLDER}/invoke_model_01.json', 'w') as file:
    json.dump(response, file, indent=4, default=str)

In [ ]:
# Extrai e salva apenas o corpo da resposta (sem metadados)
response_body = json.loads(response['body'].read())
with open(f'{OUTPUT_FOLDER}/invoke_model_02.json', 'w') as file:
    json.dump(response_body, file, indent=4, default=str)

In [ ]:
# Extrai o texto da resposta do modelo e exibe
texto_completo = response_body['output']['message']['content'][0]['text']
print(f"\nResposta do modelo:\n{texto_completo}")

# Test 2: invoke_model_with_response_stream (Streaming) 

In [ ]:
response_stream = bedrock_runtime_client.invoke_model_with_response_stream(
    body=json.dumps(body),              # Mesmo corpo da requisição anterior
    contentType='application/json',
    accept='application/json',
    modelId=MODEL_ID
)

In [ ]:
# Salva a resposta completa (metadados + conteúdo) em arquivo JSON
with open(f'{OUTPUT_FOLDER}/invoke_model_03.json', 'w') as file:
    json.dump(response_stream, file, indent=4, default=str)

In [ ]:
# Este método retorna a resposta em pedaços (chunks) conforme é gerada
response_stream = bedrock_runtime_client.invoke_model_with_response_stream(
    body=json.dumps(body),              # Mesmo corpo da requisição anterior
    contentType='application/json',
    accept='application/json',
    modelId=MODEL_ID
)
print("\nResposta em tempo real: ", end="", flush=True)

# Variável para acumular o texto completo da resposta streaming
texto_stream = ""

# Processa cada evento do stream de resposta
for event in response_stream['body']:
    # Decodifica o chunk JSON recebido
    chunk = json.loads(event['chunk']['bytes'])    
    
    # Verifica se o chunk contém texto da resposta
    if 'contentBlockDelta' in chunk:
        # Extrai o texto parcial deste chunk
        delta_text = chunk['contentBlockDelta']['delta']['text']
        # Exibe o texto imediatamente (efeito de digitação em tempo real)
        print(delta_text, end="", flush=True)
        # Acumula o texto para ter a resposta completa no final
        texto_stream += delta_text
    
    # Gera timestamp único para cada chunk (para debug/análise)
    timestamp_parcial = datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S%f')
    
    # Salva cada evento individual para análise posterior
    filename_parcial = os.path.join(OUTPUT_FOLDER, f"partial_{timestamp_parcial}.json")
    with open(filename_parcial, 'w', encoding='utf-8') as json_file:
        json.dump(event, json_file, ensure_ascii=False, indent=4, default=str)            